In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
from validation_prepare import *
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [4]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [5]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [6]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [7]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [8]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [9]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [10]:
#actions.sort_index(level = [0,2,1],inplace = True)

In [11]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [12]:
%%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

CPU times: user 10.6 s, sys: 2.49 s, total: 13.1 s
Wall time: 13.1 s


In [13]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)
movie_match_columns = {i:ii for ii,i in enumerate(match_columns)}
movie_columns_match = {ii:i for ii,i in enumerate(match_columns)}

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)

In [14]:
'''
Здесь начинается русское поле экспериментов над параметрами моделей.
'''

'\nЗдесь начинается русское поле экспериментов над параметрами моделей.\n'

In [15]:
_,_,watch_actions,_ = get_target(actions)

actions = actions.join(watch_actions['rel_dur'])
actions['rel_dur'] = actions['rel_dur'].fillna(0).replace(np.inf,1)

In [16]:
actions['action_enc'] = actions['action'].map({'watch':0,'rate':1,'bookmate':3})
actions['cons_enc'] = actions['consumption_mode'].map({'S':0,'P':1,'R':3})

In [17]:
actions.sort_index(level = [0,2,1],inplace = True)

In [18]:
#actions.reset_index()[['user_uid','element_uid']]

In [19]:
%%time
wm = actions.groupby(level = [0,1],sort = False)[['rel_dur','type','rating','action_enc','cons_enc']].mean()#.agg({'rel_dur':np.mean,'type':np.mean,'rating':np.mean,'action_enc':np.mean,})

CPU times: user 2.41 s, sys: 1.02 s, total: 3.43 s
Wall time: 3.43 s


In [20]:
wm.head()

rel_dur  type  rating  action_enc  cons_enc
user_uid element_uid                                              
0        6127         0.107821   1.0     NaN         0.0       0.0
         8168         0.025000   1.0     NaN         0.0       0.0
         1284         0.022222   1.0     NaN         0.0       0.0
         1535         0.015385   1.0     NaN         0.0       0.0
         460          0.033333   1.0     NaN         0.0       0.0

In [21]:
aa = wm.action_enc.value_counts().to_frame()
aa.columns = ['counts',]
aa['mapa'] = ['watch','bookmate','watch+rate','watch+bookmate','rate','watch+rate+bookmate','rate+bookmate']

In [22]:
wm.cons_enc.value_counts().to_frame()

,cons_enc
0.0,8296227
1.0,873834
3.0,472951


In [23]:
wm['action_enc'] = wm['action_enc'].map(aa['mapa'])
wm['cons_enc'] = wm['cons_enc'].map({0:'S',1:'P',3:'R'})

In [29]:
# Щас еще приджойним тип потребления фильма и прям ок
# И по хорошему надо бы расположить их по времени, когда их там начали потреблять
wm.loc[3]#['type'].value_counts()

,rel_dur,type,rating,action_enc,cons_enc
element_uid,,,,,
8504,0.961282,1.0,NaN,watch,S
8771,0.046460,3.0,NaN,watch,S
1380,1.007500,1.0,NaN,watch+bookmate,P
8756,0.012229,3.0,NaN,watch,S
8282,0.000000,1.0,NaN,bookmate,NaN
4205,0.116892,3.0,NaN,watch+bookmate,S
4710,1.275000,1.0,NaN,watch,P
9307,0.008929,1.0,NaN,watch,S
4473,0.015556,1.0,NaN,watch,S


In [ ]:
#%%time

#%timeit wm = actions.iloc[:1000].groupby(level = [0,1]).agg({'rel_dur':np.mean,'type':np.mean,'action':lambda x:x.nunique(),'rating':np.mean})
# Статистика по тем чувакам, которые оценивают фильмы, а еще и смотрят - типа киноманы, 
#,их мнению стоит доверять больше, наверное

In [ ]:
'''
Здесь заканчиваетс/ русское поле экспериментов
'''

In [ ]:
actions.head()

In [ ]:
#Проверим, что не загибается на полном датасете

fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,mode = 'duration',delimiter = 1/3,
                     target_col_name = 'rel_dur',)


# fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
#                        'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
#                       'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = False)
X = fe.fit_transform(actions)

In [ ]:
cfe = ColdFeatureExtractor(fe)

res_action = cfe.fit_transform(actions)

In [ ]:
res_action.keys()

In [ ]:
len(fe.train_movie_match_movie_row),len(fe.match_user_row)

In [ ]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

In [ ]:
X_user = res_action['train_user']
X_movie = res_action['train_movie']
X_user.shape,X_movie.shape

In [ ]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [ ]:


%time model.fit(X,user_features=X_user,item_features=X_movie,epochs = epochs,num_threads = num_threads,verbose = True)

In [ ]:
%%time
a,b = X.nonzero()
D = pd.DataFrame([a,b],index = ['row','columns']).T
D = D.groupby('row').agg(lambda x:frozenset(x.values))
D

In [ ]:
D.loc[1]

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
print('sparsity',X.nnz/(X.shape[0]*X.shape[1]))

In [ ]:
len( set(test_users) - set(fe.match_user_row.keys()))

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender,CosineRecommender,NearestNeighboursScorer



In [ ]:
# Для implicit  надо фильмы*юзеры матрицу.
# model = CosineRecommender()
# model.fit(X.T.tocsr())

In [ ]:
#ratings_matrix_T.shape

In [ ]:
# Сделаем какой-нибудь сабмит из lightfm
result = {}
mode = 'lightfm'#'implicit'#'lightfm'
cnt = 0
for user_uid in tqdm.tqdm(test_users):
    
    # transform user_uid to model's internal user category
    try:
        user_cat = fe.match_user_row[user_uid]
    except LookupError:
        continue
    if mode == 'lightfm':
        need_cols = np.array(list(set(np.arange(X.shape[1])) - D.get(user_cat, set())))
        recs = model.predict(user_cat,need_cols,user_features=X_user,item_features=X_movie,num_threads=num_threads)
        need_movies = np.argsort(-recs)[:20]
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i in need_movies]
    # переводим в фильмы
    
    # perform inference
    if mode == 'implicit':
        if cnt == 0:
            ratings_matrix_T = X.T.tocsr()
        recs = model.recommend(
            user_cat,
            X.tocsr(),
            N=20,
            filter_already_liked_items=True,
            filter_items=D.get(user_uid, set())
        )
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i, _ in recs]
        # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    cnt += 1

In [ ]:
user_cat

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
recs

In [ ]:
#len(need_cols),len(recs),type(recs)

In [ ]:
#plt.hist(recs)

In [ ]:
len(result)

In [ ]:

with open('answer.json', 'w') as f:
    json.dump(result, f)

In [ ]:
te.shape

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass